## Some testing and analysis of the new `Snapshot` implementation

In [1]:
import numpy as np
import mdtraj as md
import pandas as pd
import simtk.openmm as mm
from simtk.openmm import app
from simtk import unit
 
from openpathsampling.visualize import PathTreeBuilder
from IPython.display import SVG

import openpathsampling as paths

In [2]:
from IPython.display import Markdown

def code_to_md(snapshot_class):
    md = '```py\n'
    for f, s in snapshot_class.__features__['debug'].iteritems():
        md += s
        md += '\n\n'
    md += '```'

    return md

### Show the generated source code

In [3]:
EmptySnap = paths.snapshot.SnapshotFactory('no', [], 'Empty', use_lazy_reversed=False)

In [4]:
Markdown(code_to_md(EmptySnap))

```py
@staticmethod
def init_copy(self):
    self._reversed = None

def create_reversed(self):
    this = cls.__new__(cls)
    this._reversed = self
    return this

def copy_to(self, target):
    target._reversed = None

def init_empty(self):
    self._reversed = None

def copy(self):
    this = cls.__new__(cls)
    this._reversed = None
    return this

def __init__(self):
    self._reversed = None

```

In [5]:
SuperSnap = paths.snapshot.SnapshotFactory(
    'my', [
        paths.features.coordinates,
        paths.features.box_vectors,
        paths.features.velocities
    ], 'No desc', use_lazy_reversed=False)

In [6]:
Markdown(code_to_md(SuperSnap))

```py
@staticmethod
def init_copy(self, coordinates=None, box_vectors=None, velocities=None):
    self._reversed = None
    np.copyto(self.coordinates, coordinates)
    np.copyto(self.box_vectors, box_vectors)
    np.copyto(self.velocities, velocities)

def create_reversed(self):
    this = cls.__new__(cls)
    this._reversed = self
    this.coordinates = self.coordinates
    this.box_vectors = self.box_vectors
    this.velocities = - self.velocities
    return this

def copy_to(self, target):
    target._reversed = None
    np.copyto(target.coordinates, self.coordinates)
    np.copyto(target.box_vectors, self.box_vectors)
    np.copyto(target.velocities, self.velocities)

def init_empty(self):
    self._reversed = None

def copy(self):
    this = cls.__new__(cls)
    this._reversed = None
    this.coordinates = self.coordinates
    this.box_vectors = self.box_vectors
    this.velocities = self.velocities
    return this

def __init__(self, coordinates=None, box_vectors=None, velocities=None):
    self._reversed = None
    self.coordinates = coordinates
    self.box_vectors = box_vectors
    self.velocities = velocities

```

In [7]:
MegaSnap = paths.snapshot.SnapshotFactory(
    'mega', [
        paths.features.configuration,
        paths.features.momentum,
        paths.features.engine
    ], 'Long desc', use_lazy_reversed=False)

In [8]:
Markdown(code_to_md(MegaSnap))

```py
@staticmethod
def init_copy(self, configuration=None, momentum=None, is_reversed=False, engine=None):
    self._lazy = {
       cls.configuration : configuration,
       cls.momentum : momentum,
    }
    self._reversed = None
    self.is_reversed = is_reversed
    self.engine = engine

def create_reversed(self):
    this = cls.__new__(cls)
    this._lazy = {
       cls.configuration : self._lazy[cls.configuration],
       cls.momentum : self._lazy[cls.momentum],
    }
    this._reversed = self
    this.engine = self.engine
    this.is_reversed = ~ self.is_reversed
    return this

def copy_to(self, target):
    target._lazy = {
       cls.configuration : self._lazy[cls.configuration],
       cls.momentum : self._lazy[cls.momentum],
    }
    target._reversed = None
    target.is_reversed = self.is_reversed
    target.engine = self.engine

def init_empty(self):
    self._lazy = {}
    self._reversed = None

def copy(self):
    this = cls.__new__(cls)
    this._lazy = {
       cls.configuration : self._lazy[cls.configuration],
       cls.momentum : self._lazy[cls.momentum],
    }
    this._reversed = None
    this.is_reversed = self.is_reversed
    this.engine = self.engine
    return this

def __init__(self, configuration=None, momentum=None, is_reversed=False, engine=None):
    self._lazy = {
       cls.configuration : configuration,
       cls.momentum : momentum,
    }
    self._reversed = None
    self.is_reversed = is_reversed
    self.engine = engine

```

### Check the timing

In [9]:
arr = np.array(range(200000))
brr = np.array(range(200000))

In [10]:
a = SuperSnap(arr, arr, arr)

In [11]:
%%timeit
a = SuperSnap(arr, arr, arr)

The slowest run took 24.69 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 686 ns per loop


In [12]:
%%timeit
b = SuperSnap()

The slowest run took 6.83 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 593 ns per loop


In [13]:
%%timeit
b = SuperSnap(coordinates=arr, box_vectors=arr, velocities=arr)

The slowest run took 5.89 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 850 ns per loop


In [14]:
x = EmptySnap()

In [15]:
%%timeit
x = EmptySnap()

The slowest run took 9.91 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 385 ns per loop


In [16]:
%%timeit
x.reversed

The slowest run took 24.47 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 292 ns per loop


In [17]:
%%timeit
a.reversed

The slowest run took 894.47 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 291 ns per loop


In [18]:
assert(x.reversed.reversed is x)
assert(a.reversed.reversed is a)

In [19]:
%%timeit
EmptySnap.__init__(x)

The slowest run took 30.20 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 268 ns per loop


In [20]:
b = SuperSnap()

In [21]:
%%timeit
SuperSnap.__init__(b, arr, arr, arr)

The slowest run took 15.01 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 588 ns per loop


In [22]:
%%timeit
c = a.copy()

The slowest run took 6.56 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 764 ns per loop


In [23]:
%%timeit
a.copy_to(b)

The slowest run took 7.92 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.02 µs per loop


In [24]:
%%timeit
SuperSnap.init_copy(a, arr, arr, arr)

The slowest run took 22.58 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 929 ns per loop


In [25]:
%%timeit
SuperSnap.init_empty(a)

The slowest run took 11.33 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 252 ns per loop
